In [645]:
from osgeo import gdal, osr,ogr
import numpy as np
import json
import sys
import os
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt
import cv2

In [646]:
path_to_spacenet_utils = 'python_scripts_sn/'
sys.path.extend([path_to_spacenet_utils])
from spaceNetUtilities import geoTools as gT

In [647]:
def geojson_to_pixel_arr(raster_file, geojson_file, pixel_ints=True,
                       verbose=False):
    '''
    Tranform geojson file into array of points in pixel (and latlon) coords
    pixel_ints = 1 sets pixel coords as integers
    '''
    
    # load geojson file
    with open(geojson_file) as f:
        geojson_data = json.load(f)

    # load raster file and get geo transforms
    src_raster = gdal.Open(raster_file)
    targetsr = osr.SpatialReference()
    targetsr.ImportFromWkt(src_raster.GetProjectionRef())
        
    geom_transform = src_raster.GetGeoTransform()
    
    # get latlon coords
    latlons = []
    types = []
    for feature in geojson_data['features']:
        coords_tmp = feature['geometry']['coordinates'][0]
        type_tmp = feature['geometry']['type']
        if verbose: 
            print("features:", feature.keys())
            print("geometry:features:", feature['geometry'].keys())

            #print "feature['geometry']['coordinates'][0]", z
        latlons.append(coords_tmp)
        types.append(type_tmp)
        #print feature['geometry']['type']
    
    # convert latlons to pixel coords
    pixel_coords = []
    latlon_coords = []
    for i, (poly_type, poly0) in enumerate(zip(types, latlons)):
        
        if poly_type.upper() == 'MULTIPOLYGON':
            #print "oops, multipolygon"
            for poly in poly0:
                poly=np.array(poly)
                if verbose:
                    print("poly.shape:", poly.shape)
                    
                # account for nested arrays
                if len(poly.shape) == 3 and poly.shape[0] == 1:
                    poly = poly[0]
                    
                poly_list_pix = []
                poly_list_latlon = []
                if verbose: 
                    print("poly", poly)
                for coord in poly:
                    if verbose: 
                        print("coord:", coord)
                    lon, lat, z = coord 
                    px, py = gT.latlon2pixel(lat, lon, input_raster=src_raster,
                                         targetsr=targetsr, 
                                         geom_transform=geom_transform)
                    poly_list_pix.append([px, py])
                    if verbose:
                        print("px, py", px, py)
                    poly_list_latlon.append([lat, lon])
                
                if pixel_ints:
                    ptmp = np.rint(poly_list_pix).astype(int)
                else:
                    ptmp = poly_list_pix
                pixel_coords.append(ptmp)
                latlon_coords.append(poly_list_latlon)            

        elif poly_type.upper() == 'POLYGON':
            poly=np.array(poly0)
            if verbose:
                print("poly.shape:", poly.shape)
                
            # account for nested arrays
            if len(poly.shape) == 3 and poly.shape[0] == 1:
                poly = poly[0]
                
            poly_list_pix = []
            poly_list_latlon = []
            if verbose: 
                print("poly", poly)
            for coord in poly:
                if verbose: 
                    print("coord:", coord)
                lon, lat, z = coord 
                px, py = gT.latlon2pixel(lat, lon, input_raster=src_raster,
                                     targetsr=targetsr, 
                                     geom_transform=geom_transform)
                poly_list_pix.append([px, py])
                if verbose:
                    print("px, py", px, py)
                poly_list_latlon.append([lat, lon])
            
            if pixel_ints:
                ptmp = np.rint(poly_list_pix).astype(int)
            else:
                ptmp = poly_list_pix
            pixel_coords.append(ptmp)
            latlon_coords.append(poly_list_latlon)
            
        else:
            print("Unknown shape type in coords_arr_from_geojson()")
            return
            
    return pixel_coords, latlon_coords

In [648]:
def plot_truth_coords(input_image, pixel_coords,   
                  figsize=(8,8), plot_name='',
                  add_title=False, poly_face_color='orange', 
                  poly_edge_color='red', poly_nofill_color='blue', cmap='bwr'):
    '''Plot ground truth coordinaates, pixel_coords should be a numpy array'''
    
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(2*figsize[0], figsize[1]))
    
    if add_title:
        suptitle = fig.suptitle(plot_name.split('/')[-1], fontsize='large')
    
    # create patches
    patches = []
    patches_nofill = []
    
    if len(pixel_coords) > 0:
        # get patches    
        for coord in pixel_coords:
            patches_nofill.append(Polygon(coord, facecolor=poly_nofill_color, 
                                          edgecolor=poly_edge_color, lw=3))
            patches.append(Polygon(coord, edgecolor=poly_edge_color, fill=True, 
                                   facecolor=poly_face_color))
        #p0 = PatchCollection(patches, alpha=0.25, match_original=True)
        p0 = PatchCollection(patches, alpha=0.60, match_original=True)
        p2 = PatchCollection(patches_nofill, alpha=0.75, match_original=True)
        
                   
    # ax0: raw image
   
    ax0.imshow(input_image)
    print(input_image)
    if len(patches) > 0:
        ax0.add_collection(p0)
    ax0.set_title('Input Image + Ground Truth Buildings') 
    
    # truth polygons
    zero_arr = np.zeros(input_image.shape[:2])
    # set background to white?
    #zero_arr[zero_arr == 0.0] = np.nan
    ax1.imshow(zero_arr, cmap=cmap)
    if len(patches) > 0:
        ax1.add_collection(p2)
    ax1.set_title('Ground Truth Building Polygons')
        
    #plt.axis('off')
    plt.tight_layout()
    if add_title:
        suptitle.set_y(0.95)
        fig.subplots_adjust(top=0.96)
    #plt.show()
 
    if len(plot_name) > 0:
        plt.savefig(plot_name)
    
    return patches, patches_nofill

In [649]:
def create_building_mask(rasterSrc, vectorSrc, npDistFileName='', 
                            noDataValue=0, burn_values=1):

    '''
    Create building mask for rasterSrc,
    Similar to labeltools/createNPPixArray() in spacenet utilities
    '''
    
    ## open source vector file that truth data
    source_ds = ogr.Open(vectorSrc)
    source_layer = source_ds.GetLayer()

    ## extract data from src Raster File to be emulated
    ## open raster file that is to be emulated
    srcRas_ds = gdal.Open(rasterSrc)
    cols = srcRas_ds.RasterXSize
    rows = srcRas_ds.RasterYSize

    ## create First raster memory layer, units are pixels
    # Change output to geotiff instead of memory 
    memdrv = gdal.GetDriverByName('GTiff') 
    dst_ds = memdrv.Create(npDistFileName, cols, rows, 1, gdal.GDT_Byte, options=['COMPRESS=LZW'])
    dst_ds.SetGeoTransform(srcRas_ds.GetGeoTransform())
    dst_ds.SetProjection(srcRas_ds.GetProjection())
    band = dst_ds.GetRasterBand(1)
    band.SetNoDataValue(noDataValue)    
    gdal.RasterizeLayer(dst_ds, [1], source_layer, burn_values=[burn_values])
    dst_ds = 0
    
    return band

In [650]:
def plot_building_mask(input_image, pixel_coords, mask_image,   
                  figsize=(8,8), plot_name='',
                  add_title=False, poly_face_color='orange', 
                  poly_edge_color='red', poly_nofill_color='blue', cmap='bwr'):


    fig, (ax0, ax1, ax2) = plt.subplots(1, 3, 
                                        figsize=(3*figsize[0], figsize[1]))
    
    if add_title:
        suptitle = fig.suptitle(plot_name.split('/')[-1], fontsize='large')

    # create patches
    patches = []
    patches_nofill = []
    if len(pixel_coords) > 0:
        # get patches    
        for coord in pixel_coords:
            patches_nofill.append(Polygon(coord, facecolor=poly_nofill_color, 
                                          edgecolor=poly_edge_color, lw=3))
            patches.append(Polygon(coord, edgecolor=poly_edge_color, fill=True, 
                                   facecolor=poly_face_color))

        p0 = PatchCollection(patches, alpha=0.60, match_original=True)
        p1 = PatchCollection(patches_nofill, alpha=0.75, match_original=True)
        
    #if len(patches) > 0:
    #    p0 = PatchCollection(patches, alpha=0.25, match_original=True)
    #    #p1 = PatchCollection(patches, alpha=0.75, match_original=True)
    #    p1 = PatchCollection(patches_nofill, alpha=0.75, match_original=True)                   
 
    # ax0: raw image
    
    ax0.imshow(input_image)
    if len(patches) > 0:
        ax0.add_collection(p0)
    ax0.set_title('Input Image + Ground Truth Buildings') 
    
    # truth polygons
    zero_arr = np.zeros(input_image.shape[:2])
    # set background to white?
    #zero_arr[zero_arr == 0.0] = np.nan
    ax1.imshow(zero_arr, cmap=cmap)
    if len(patches) > 0:
        ax1.add_collection(p1)
    ax1.set_title('Ground Truth Building Polygons')
        
    # old method of truth, with mask
    ## ax0: raw imageø
    #ax0.imshow(input_image)
    ## ground truth
    ## set zeros to nan
    #palette = plt.cm.gray
    #palette.set_over('orange', 1.0)
    #z = mask_image.astype(float)
    #z[z==0] = np.nan
    #ax0.imshow(z, cmap=palette, alpha=0.25, 
    #        norm=matplotlib.colors.Normalize(vmin=0.5, vmax=0.9, clip=False))
    #ax0.set_title('Input Image + Ground Truth Buildings') 
   
    # mask
    mask_image=mask_image.astype(float)
    ax2.imshow(mask_image, cmap=cmap)
    # truth polygons?
    #if len(patches) > 0:
    #    ax1.add_collection(p1)
    ax2.set_title('Ground Truth Building Mask')    
          
    #plt.axis('off')
    plt.tight_layout()
    if add_title:
        suptitle.set_y(0.95)
        fig.subplots_adjust(top=0.96)
    #plt.show()
 
    if len(plot_name) > 0:
        plt.savefig(plot_name)
    
    return

In [651]:
if __name__ == '__main__':
    
    #script_test_functions for spacenet
#     path='/scratch/SPACENET_DATA/BUILDING_DATASET/processedBuildingLabels/'
#     geojson_file=path+'vectordata/geojson/Geo_AOI_1_RIO_img5142.geojson'
#     raster_file=path+'3band/3band_AOI_1_RIO_img5142.tif'
#     save_folder='test_preprocess_spacenet/'
#     pixel_coords, latlon_coords=geojson_to_pixel_arr(raster_file, geojson_file, True,False)
#     img_input = cv2.imread(raster_file)
#     patches, patches_nofill=plot_truth_coords(img_input, pixel_coords,figsize=(8,8), plot_name=save_folder+'buildings_frame_patch_5142',
#                                               add_title=True, poly_face_color='orange', poly_edge_color='red', 
#                                               poly_nofill_color='blue', cmap='bwr')
    
#     create_building_mask(raster_file, geojson_file,npDistFileName=save_folder+'mask_img5142')
#     mask_img = cv2.imread(save_folder+'mask_img5142')
#     plot_building_mask(img_input, pixel_coords, mask_img,   
#                   figsize=(8,8), plot_name=save_folder+'buildings_mask_patch_5142',
#                   add_title=False, poly_face_color='orange', 
#                   poly_edge_color='red', poly_nofill_color='blue', cmap='bwr')
    
    #script_test_functions for data ghana
    
    path='/home/s161362/bckup_themis/DATA_GHANA/'
    geojson_file=path+'buildings_accra.geojson'
    raster_file=path+'Panchromatic_img.tif'
    save_folder='test_preprocess_ghana/'
    img_input = cv2.imread(raster_file)
    pixel_coords, latlon_coords=geojson_to_pixel_arr(raster_file, geojson_file, True,True)
#     patches, patches_nofill=plot_truth_coords(img_input, pixel_coords,figsize=(8,8), plot_name=save_folder+'buildings_frame',
#                                               add_title=True, poly_face_color='orange', poly_edge_color='red', 
#                                               poly_nofill_color='blue', cmap='bwr')
    
#     create_building_mask(raster_file, geojson_file,npDistFileName=save_folder+'mask_img5142')
#     mask_img = cv2.imread(save_folder+'mask_img')
#     plot_building_mask(img_input, pixel_coords, mask_img,   
#                   figsize=(8,8), plot_name=save_folder+'buildings_mask',
#                   add_title=False, poly_face_color='orange', 
#                   poly_edge_color='red', poly_nofill_color='blue', cmap='bwr')
    



features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: 

features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: 

features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: dict_keys(['type', 'properties', 'geometry'])
geometry:features: dict_keys(['type', 'coordinates'])
features: 

px, py -346808.9098214124 1249350.4301940254
coord: [179993.09328729 631683.37428882      0.        ]
px, py -346809.05578257504 1249350.0905771046
coord: [180010.03359841 631689.20379905      0.        ]
px, py -346808.4644541643 1249349.88708881
coord: [180014.64394622 631680.73889939      0.        ]
px, py -346808.3035226663 1249350.1825695501
poly.shape: (6, 3)
poly [[180013.67283653 631702.37109207      0.        ]
 [180006.04828296 631699.4523152       0.        ]
 [180003.13318283 631708.753451        0.        ]
 [180011.5972673  631710.40178807      0.        ]
 [180013.27228522 631707.0166322       0.        ]
 [180013.67283653 631702.37109207      0.        ]]
coord: [180013.67283653 631702.37109207      0.        ]
px, py -346808.3374207894 1249349.427463577
coord: [180006.04828296 631699.4523152       0.        ]
px, py -346808.60356791684 1249349.5293481096
coord: [180003.13318283 631708.753451        0.        ]
px, py -346808.7053241076 1249349.2046772218
coord: [18001

coord: [179573.41994192 632442.72148209      0.        ]
px, py -346823.7051458941 1249323.584359728
coord: [179575.40784768 632442.61075289      0.        ]
px, py -346823.63575489266 1249323.5882249065
coord: [179571.98995075 632418.11371731      0.        ]
px, py -346823.75506200263 1249324.443332762
coord: [179570.80060878 632418.31958147      0.        ]
px, py -346823.796577869 1249324.4361467473
coord: [179570.13786371 632412.75589409      0.        ]
px, py -346823.8197120363 1249324.6303560738
coord: [179565.24306225 632413.08662392      0.        ]
px, py -346823.99057283957 1249324.6188114248
coord: [179565.4631152  632419.27082922      0.        ]
px, py -346823.98289154266 1249324.4029419366
coord: [179560.01753097 632419.30434469      0.        ]
px, py -346824.1729782917 1249324.4017720262
coord: [179563.65555908 632446.62262849      0.        ]
px, py -346824.0459871541 1249323.4481840294
coord: [179573.57745392 632445.43200111      0.        ]
px, py -346823.699647688

...

px, py -346851.86352711264 1249299.6678543503
poly.shape: (5, 3)
poly [[179159.22535842 633342.15642994      0.        ]
 [179172.12761353 633342.15642994      0.        ]
 [179171.24791431 633329.98725752      0.        ]
 [179158.63889228 633330.28049058      0.        ]
 [179159.22535842 633342.15642994      0.        ]]
coord: [179159.22535842 633342.15642994      0.        ]
px, py -346838.163264346 1249292.188157233
coord: [179172.12761353 633342.15642994      0.        ]
px, py -346837.712890681 1249292.1881572332
coord: [179171.24791431 633329.98725752      0.        ]
px, py -346837.7435979764 1249292.6129414851
coord: [179158.63889228 633330.28049058      0.        ]
px, py -346838.1837358763 1249292.60270572
coord: [179159.22535842 633342.15642994      0.        ]
px, py -346838.163264346 1249292.188157233
poly.shape: (5, 3)
poly [[179227.25543079 633318.6977843       0.        ]
 [179243.0900166  633317.23161895      0.        ]
 [179242.79678353 633305.79552919      0.    

px, py -346841.1851177384 1249311.9696051378
coord: [179077.3966582  632768.50109343      0.        ]
px, py -346841.0196248291 1249312.2125060207
coord: [179073.64972097 632765.9011778       0.        ]
px, py -346841.1504176122 1249312.3032601967
coord: [179069.06163456 632773.01271173      0.        ]
px, py -346841.31057204056 1249312.0550208327
coord: [179072.65563558 632775.45969114      0.        ]
px, py -346841.1851177384 1249311.9696051378
poly.shape: (5, 3)
poly [[179124.59422722 632744.55554818      0.        ]
 [179130.7854896  632738.04678516      0.        ]
 [179122.84797372 632731.69677246      0.        ]
 [179116.65671134 632738.04678516      0.        ]
 [179124.59422722 632744.55554818      0.        ]]
coord: [179124.59422722 632744.55554818      0.        ]
px, py -346839.3721188725 1249313.0483632323
coord: [179130.7854896  632738.04678516      0.        ]
px, py -346839.15600304573 1249313.275561922
coord: [179122.84797372 632731.69677246      0.        ]
px, p

px, py -346821.23237775516 1249299.022488295
coord: [179650.08037593 633144.25059622      0.        ]
px, py -346821.02919193497 1249299.096374048
coord: [179648.49287276 633140.811006        0.        ]
px, py -346821.0846062496 1249299.2164383961
coord: [179642.40744392 633141.34017373      0.        ]
px, py -346821.2970277888 1249299.197966958
coord: [179644.25953096 633146.36726712      0.        ]
px, py -346821.23237775516 1249299.022488295
poly.shape: (6, 3)
poly [[179322.12922958 633299.23346         0.        ]
 [179334.67050466 633300.97971349      0.        ]
 [179334.9880053  633289.70844095      0.        ]
 [179329.48466096 633289.12635645      0.        ]
 [179321.65297863 633289.54969063      0.        ]
 [179322.12922958 633299.23346         0.        ]]
coord: [179322.12922958 633299.23346         0.        ]
px, py -346832.47684651415 1249293.6864515333
coord: [179334.67050466 633300.97971349      0.        ]
px, py -346832.039073429 1249293.6254957875
coord: [17933

px, py -346702.18851796695 1249290.4119465689
coord: [183057.39303292 633393.43799303      0.        ]
px, py -346702.09154291643 1249290.3980929903
coord: [183057.26074099 633387.48485612      0.        ]
px, py -346702.09616077604 1249290.60589667
coord: [183049.72010091 633384.70672556      0.        ]
px, py -346702.3593787703 1249290.7028717205
coord: [183046.54509456 633392.24736565      0.        ]
px, py -346702.47020739946 1249290.439653726
coord: [183048.92634932 633394.09945269      0.        ]
px, py -346702.3870859276 1249290.3750036925
poly.shape: (5, 3)
poly [[182892.72265149 633265.10159052      0.        ]
 [182896.53265911 633260.97408227      0.        ]
 [182892.40515086 633257.32282497      0.        ]
 [182888.43639292 633261.45033322      0.        ]
 [182892.72265149 633265.10159052      0.        ]]
coord: [182892.72265149 633265.10159052      0.        ]
px, py -346707.8396235892 1249294.8778785379
coord: [182896.53265911 633260.97408227      0.        ]
px, p

px, py -346792.04385884025 1249353.8911873759
poly.shape: (5, 3)
poly [[180134.67493644 631514.50991486      0.        ]
 [180140.28411433 631519.16659085      0.        ]
 [180142.40078523 631514.93324904      0.        ]
 [180137.95577634 631511.22907497      0.        ]
 [180134.67493644 631514.50991486      0.        ]]
coord: [180134.67493644 631514.50991486      0.        ]
px, py -346804.1136506988 1249355.9850557346
coord: [180140.28411433 631519.16659085      0.        ]
px, py -346803.91785345395 1249355.8225070783
coord: [180142.40078523 631514.93324904      0.        ]
px, py -346803.8439677011 1249355.970278584
coord: [180137.95577634 631511.22907497      0.        ]
px, py -346803.99912778195 1249356.0995786514
coord: [180134.67493644 631514.50991486      0.        ]
px, py -346804.1136506988 1249355.9850557346
poly.shape: (5, 3)
poly [[180167.06000121 631873.62077197      0.        ]
 [180175.42085127 631878.91244923      0.        ]
 [180177.22002153 631874.36160679    